# CHALLENGE LATAM AIRLINES - GLOBANT : Flight delay prediction

## Setup and ETL's

In this section we will setup our environment and also will get  all the data needed to make our EDA, this include internal and external databases. Also, we will create some synthetic features that we think could be usefull to better understand the patterns associated with flight delays

### Setup environment

#### Import libraries

In [1]:
import os
import sys
from datetime import datetime, timedelta
import pandas as pd
# Set some Pandas options
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 50)
import numpy as np
import statsmodels as sm
from scipy.stats import chi2
import matplotlib.pyplot as plt
import matplotlib_inline.backend_inline
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
from sklearn import preprocessing

#### Set environment

In [2]:
directory_path = os.path.abspath(os.path.join('..'))
if directory_path not in sys.path:
    sys.path.append(directory_path)
os.chdir(directory_path)
os.getcwd()

'/Users/iairlinker/Documents/repos/flight_delays_challenge'

### Home made functions definition

In [3]:
def get_hist(var1,var2,name_image,values_range=[0,1],nbins=20,normed=True):
    
    common_params = dict(bins=nbins,range=values_range,normed=normed)
    plt.subplots_adjust(hspace=.8)
    plt.subplot(311)
    plt.title('Default')
    #plt.hist(df.popularity, **common_params)
    plt.hist(var1, **common_params)
    plt.hist(var2, **common_params)
    plt.subplot(312)
    plt.title('Skinny shift - 2 at a time')
    plt.hist((var1, var2), **common_params)
    plt.subplot(313)
    common_params['histtype'] = 'step'
    plt.title('With steps')
    #plt.hist(df.popularity, **common_params)
    plt.hist(var1, **common_params)
    plt.hist(var2, **common_params)

    plt.savefig(name_image)
    plt.show()

def get_period_day(x):
    if((x>=5) & (x<12)):
        return 'morning'
    elif((x>=12) & (x<19)):
        return 'afternoon'
    elif((x>=19) or (x<5)):
        return 'night'
    
def is_high_season(x):
    if( (x <= datetime(2017, 3, 3).date()) or  (x >= datetime(2017, 12, 15).date())):
        return 1
    elif( (x >= datetime(2017, 7, 15).date()) or  (x >= datetime(2017, 7, 31).date())):
        return 1
    elif( (x >= datetime(2017, 9, 11).date()) or  (x >= datetime(2017, 9, 30).date())):
        return 1
    else:
        return 0

### Get the flightdata , generate the target variable and the synthetic features asked in the instructions

In [4]:
path = os.getcwd() + '/data/raw/dataset_SCL.csv'
flights = pd.read_csv(path,low_memory=False)
print(flights.shape)
flights.head()

(68206, 18)


,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,DIA,MES,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,1,1,2017,Domingo,I,American Airlines,Santiago,Miami
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,2,1,2017,Lunes,I,American Airlines,Santiago,Miami
2,2017-01-03 23:30:00,226,SCEL,KMIA,AAL,2017-01-03 23:39:00,226,SCEL,KMIA,AAL,3,1,2017,Martes,I,American Airlines,Santiago,Miami
3,2017-01-04 23:30:00,226,SCEL,KMIA,AAL,2017-01-04 23:33:00,226,SCEL,KMIA,AAL,4,1,2017,Miercoles,I,American Airlines,Santiago,Miami
4,2017-01-05 23:30:00,226,SCEL,KMIA,AAL,2017-01-05 23:28:00,226,SCEL,KMIA,AAL,5,1,2017,Jueves,I,American Airlines,Santiago,Miami


In [5]:
flights['Fecha-O'] = pd.to_datetime(flights['Fecha-O'])
flights['Fecha-I'] = pd.to_datetime(flights['Fecha-I'])
ind = flights[flights['Fecha-O']>flights['Fecha-I']].index
flights.loc[ind,'dif_min'] = (flights.loc[ind]['Fecha-O']-flights.loc[ind]['Fecha-I']).apply(lambda x: (x.seconds)/60)
ind = flights[flights['Fecha-I']>flights['Fecha-O']].index
flights.loc[ind,'dif_min'] = (flights.loc[ind]['Fecha-I']-flights.loc[ind]['Fecha-O']).apply(lambda x: -1* (x.seconds)/60)
flights['atraso_15'] = flights.dif_min.apply(lambda x: 1 if (x>15) else 0)
flights['periodo_dia'] = flights['Fecha-I'].apply(lambda x : get_period_day(x.hour))
flights['temporada_alta'] = flights['Fecha-I'].apply(lambda x : is_high_season(x.date()))
path = os.getcwd() + '/data/processed/synthetic_features.csv' 
flights.to_csv(path,
          sep=',',
          index = False,
          columns = ['temporada_alta','dif_min','atraso_15','periodo_dia']
         )
flights.head(2)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,AÑO,DIANOM,TIPOVUELO,OPERA,SIGLAORI,SIGLADES,dif_min,atraso_15,periodo_dia,temporada_alta
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,2017,Domingo,I,American Airlines,Santiago,Miami,3.0,0,night,1
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,2017,Lunes,I,American Airlines,Santiago,Miami,9.0,0,night,1


### Check airports and add external data aggregation

#### Check departure and arrival airports

First, we check how many departure airports I have, how many arrivals and to understand the number of possible combinations between them, which will allow me to have an idea of the complexity that this variable will add and design strategies to address it.

In [6]:
len(flights['Des-I'].unique())

64

In [7]:
'SCEL' in flights['Des-I'].unique()

False

In [8]:
flights['Ori-I'].unique()

array(['SCEL'], dtype=object)

#### Add the International Airport Database

Then, we agreggate the data from the Global Airport database (<https://www.partow.net/miscellaneous/airportdatabase/>) which will give me the name of the airport, country where is located, the latitud and longitud for each airport. This variables will give me the chance to create some new features like distance between origin and destiny and , if i had time, get the wather in the airport for each flyght.

In [9]:
col_names = ['ICAO Code','IATA Code','Airport Name','City/Town','Country','Lat','Long']
path = os.getcwd() + '/data/external/GlobalAirportDatabase.txt'
airports = pd.read_csv(path,
                       sep=':',
                       usecols = [0,1,2,3,4,14,15],
                       header = None,
                       names = col_names
                      )
print(airports.shape)
airports.head(2)

(9300, 7)


,ICAO Code,IATA Code,Airport Name,City/Town,Country,Lat,Long
0,AYGA,GKA,GOROKA,GOROKA,PAPUA NEW GUINEA,-6.082,145.392
1,AYLA,LAE,NaN,LAE,PAPUA NEW GUINEA,0.000,0.000


We check that all the origin and destination codes are in the airport database, and if this is not the case, we will look for mechanisms to leave the column that has the ICAO codes from our flights database

In [10]:
for element in flights['Des-I'].unique():
    if(element not in airports['ICAO Code'].unique()):
        print('Aún tienes códigos ICAO distintos para el mismo aerpuerto y es: '+ str(element))

Aún tienes códigos ICAO distintos para el mismo aerpuerto y es: SAEZ
Aún tienes códigos ICAO distintos para el mismo aerpuerto y es: SPJC
Aún tienes códigos ICAO distintos para el mismo aerpuerto y es: SCNT
Aún tienes códigos ICAO distintos para el mismo aerpuerto y es: SCQP
Aún tienes códigos ICAO distintos para el mismo aerpuerto y es: SCAT
Aún tienes códigos ICAO distintos para el mismo aerpuerto y es: SCPQ
Aún tienes códigos ICAO distintos para el mismo aerpuerto y es: SEQM


This information of this website <https://www.world-airport-codes.com> was used to correct the airports codes that does not match. We just get the ICAO code that wasn't in 
the aiiports database and get the IATA code of the airport. Then, changed the original ICAO code for the one of our database.

In the case of UIO we had to leave the ICAO code SEQU in the old location of Mariscal Sucre International Airport, since this one is now closed but our flyght database has arrivals on there and also in the new one, which is SEQM, the one which we create with the new location above

In [11]:
airports.loc[airports['IATA Code'] == 'EZE','ICAO Code'] = 'SAEZ' # Ezeiza International airport of Buenos Aires, Argentina,
airports.loc[airports['IATA Code'] == 'LIM','ICAO Code'] = 'SPJC' # intenational airport of Lima, Perú
airports.loc[airports['IATA Code'] == 'ZCO','ICAO Code'] = 'SCQP' # National airport ofAraucania, Chile
airports.loc[airports['IATA Code'] == 'CPO','ICAO Code'] = 'SCAT' # National airport of Calama, Chile
airports.loc[airports['IATA Code'] == 'UIO','ICAO Code'] = 'SEQU' # International airport Mariscal Sucre Quito, Ecuador

Also, we had to create two airports that wasn't in any of the databases and add it to ours, both are new airports builded in remote areas of Chile

In [12]:
uio_airport = {'ICAO Code':'SEQM', 
           'IATA Code':'UIO', 
           'Airport Name':'MARISCAL SUCRE INTERNATIONAL', 
           'City/Town':'QUITO',
           'Country':'ECUADOR',
           'Lat':-0.1291667,
           'Long':-78.3575
          }

pnt_airport = {'ICAO Code':'SCNT', 
           'IATA Code':'PNT', 
           'Airport Name':'TENIENTE JULIO GALLARDO', 
           'City/Town':'PUERTO NATALES',
           'Country':'CHILE',
           'Lat':-51.6715012,
           'Long':-72.5283966
          }
mhc_airport = {'ICAO Code':'SCPQ', 
           'IATA Code':'MHC', 
           'Airport Name':'MOCOPULLI AIRPORT', 
           'City/Town':'DALCAHUE',
           'Country':'CHILE',
           'Lat':-42.34611,
           'Long':-73.71389
          }
pnt_airport = pd.DataFrame.from_dict(pnt_airport, orient = 'index').T
mhc_airport = pd.DataFrame.from_dict(mhc_airport, orient = 'index').T
uio_airport = pd.DataFrame.from_dict(uio_airport, orient = 'index').T
#append row to the dataframe
airports = pd.concat([airports,mhc_airport,uio_airport,pnt_airport],ignore_index = True)

In [13]:
for element in flights['Des-I'].unique():
    if(element not in airports['ICAO Code'].unique()):
        print('Aún tienes códigos ICAO distintos para el mismo aerpuerto y es: '+ str(element))

In [14]:
df = pd.merge(flights, airports, left_on = 'Des-I',right_on = 'ICAO Code', how='left')
path = os.getcwd() + '/data/processed/database.csv' 
df.to_csv(path,
          sep=',',
          index = False
         )
df.head(2)

,Fecha-I,Vlo-I,Ori-I,Des-I,Emp-I,Fecha-O,Vlo-O,Ori-O,Des-O,Emp-O,...,atraso_15,periodo_dia,temporada_alta,ICAO Code,IATA Code,Airport Name,City/Town,Country,Lat,Long
0,2017-01-01 23:30:00,226,SCEL,KMIA,AAL,2017-01-01 23:33:00,226,SCEL,KMIA,AAL,...,0,night,1,KMIA,MIA,MIAMI INTERNATIONAL,MIAMI,USA,25.793,-80.291
1,2017-01-02 23:30:00,226,SCEL,KMIA,AAL,2017-01-02 23:39:00,226,SCEL,KMIA,AAL,...,0,night,1,KMIA,MIA,MIAMI INTERNATIONAL,MIAMI,USA,25.793,-80.291


### Conclusions

* All the flights depart from the Arturo Merino Benitez Airport in Santiago, Chile and arrive at 64 different airports. This limits the complexity of the problem, since you only have to understand the incidence of 64 possible combinations

* We where aible to correct manually the seven ICAO codes in the Global Airport database that doesn't match our database. Probably, the updated code is the one from the external database, but since our model will be tested with the ICAO codes given we preserved the ones from 2017. After that, we added the some information of the external database to ours in order to understand the influence of the location of the airport and the distance between departure and arrival airports in the delays

* Personalized functions where created to generate the sythetic features asked in the instructions and it was saved in the data processed file

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Exploratory data analysis on the variables

On this section we are going to make an exploratory data analysis following this steps:

* Check Data integrity
* Understand distribution of the data for each variable
* Check this hypothesis
* Multivariable analysis
* Conclusions

#### Check Data integrity
* missing values
* Check outliers
* Check out of range values

#### Understand distribution of the data for each variable

* Check the distribution of the delays in minutes (is there extreme values?)
* Check the distribution of the delays as binary variable (it's balanced?)
* Check the distribution of the explanatory variables (are they evenly distributed?) 
* Check how much of the arrival airport planned is the same as the arrival airport operated
* Check how much of the  planned airline is the same as the operated airline
* Check how much of the planned flight number is the same as the operated flight number

#### Check this hypothesis

* There is a significance difference in delays between airlines
* There is a significance difference in delays between airports
* There is a significance difference in delays between the type of flyght (international or national)
* There is a significance difference in delays between high season and the rest of the year
* There is a significance difference in delays between the 3 differents periods of the day (morning, afternoon and night)
* There is a strong correlation between distance between arrival and departure airport and delays

#### Multivariable analysis

* Understand the relation of the explanatory variables with the target
* Understand the relation of the explanatory variables between them

#### Conclusions

* Summary of the main insights
* Explanation of which variables are the best candidates to be selected for the model and why



----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------